# Final

- Complete pipeline for Visual Similarity based Fashion Item Recommendation Engine

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import resnet
import matplotlib.pyplot as plt
from skimage import io
%cd .\yolov5
from detect import generate_bbox
%matplotlib inline

[WinError 2] The system cannot find the file specified: '.\\yolov5'
C:\Users\CrimsonX\Shop-The-Look-main


ModuleNotFoundError: No module named 'detect'

In [ ]:
LOADED_MODELS = dict()
LOADED_CSVS = dict()
TARGET_SHAPE = (224,224,3)

In [ ]:
def preprocess_image1(filename):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape for Gender Classification
    """
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.resize(image, TARGET_SHAPE[:2])
    return tf.expand_dims(image, axis=0)

def preprocess_image2(filename):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape for Embedding Generation
    """
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, TARGET_SHAPE[:2])
    image = resnet.preprocess_input(image)
    return tf.expand_dims(image, axis=0)

def load_models():
    """
    Load Various Models for Recommendation Engine Pipeline
    """
    if len(LOADED_MODELS) == 4:
        return True
    else:
        print("Loading Models...")
        GENDER_CLASSIFIER = load_model("..\\MODELS\\gender_classification_model.h5")
        TOPWEAR_EMBEDDING = load_model("..\\MODELS\\topwear_embedding.h5")
        BOTTOMWEAR_EMBEDDING = load_model("..\\MODELS\\bottomwear_embedding.h5")
        FOOTWEAR_EMBEDDING = load_model("..\\MODELS\\footwear_embedding.h5")
        
        LOADED_MODELS["gender_classifier"] = GENDER_CLASSIFIER
        LOADED_MODELS["topwear"] = TOPWEAR_EMBEDDING
        LOADED_MODELS["bottomwear"] = BOTTOMWEAR_EMBEDDING
        LOADED_MODELS["footwear"] = FOOTWEAR_EMBEDDING
        print("Models Loaded!")
        return True
    
def load_CSVs():
    """
    Load Various CSV files for Embedding Generation Pipeline
    """
    if len(LOADED_CSVS) == 6:
        return True
    else:
        print("Loading CSVs...")
        MENS_TOPWEAR_CSV = pd.read_csv("..\\CSVS\\mens_topwear_embeddings.csv")
        MENS_BOTTOMWEAR_CSV = pd.read_csv("..\\CSVS\\mens_bottomwear_embeddings.csv")
        MENS_FOOTWEAR_CSV = pd.read_csv("..\\CSVS\\mens_footwear_embeddings.csv")
        WOMENS_TOPWEAR_CSV = pd.read_csv("..\\CSVS\\womens_topwear_embeddings.csv")
        WOMENS_BOTTOMWEAR_CSV = pd.read_csv("..\\CSVS\\womens_bottomwear_embeddings.csv")
        WOMENS_FOOTWEAR_CSV = pd.read_csv("..\\CSVS\\womens_footwear_embeddings.csv")
        
        LOADED_CSVS["mens_topwear"] = MENS_TOPWEAR_CSV
        LOADED_CSVS["mens_bottomwear"] = MENS_BOTTOMWEAR_CSV
        LOADED_CSVS["mens_footwear"] = MENS_FOOTWEAR_CSV
        LOADED_CSVS["womens_topwear"] = WOMENS_TOPWEAR_CSV
        LOADED_CSVS["womens_bottomwear"] = WOMENS_BOTTOMWEAR_CSV
        LOADED_CSVS["womens_footwear"] = WOMENS_FOOTWEAR_CSV
        print("CSVs Loaded!")
        return True
    
def extract_clothes(image): 
    '''
    Extract Topwear, Bottomwear and Footwear from a given Image
    Note: In case there are multiple items of the same class, the item with highest confidence score is returned.
    
    Args:
        image: Path to the image file that you want to extract clothes from.
        
    Returns:
        ouputs: Dict[numpy.ndarray], Dictionary containing detected Object Class as Keys and the image slice as Values. Also contains the original image.
    '''
    temporary_image_storage = False
    if isinstance(image, str):
        img_path = image
        image = io.imread(img_path)  
    elif isinstance(image, np.ndarray):
        img_path = os.path.join("temp","example.jpg")
        saved_image = Image.fromarray(image)
        saved_image.save(img_path)
        temporary_image_storage = True
    results = generate_bbox(img_path, conf_thres=0.3)
    if temporary_image_storage:
        os.remove(img_path)
    topwear_score, bottomwear_score, footwear_score = 0, 0, 0
    outputs = {"original_image": image}
    for result in results[0]:
        XMIN, YMIN, XMAX, YMAX = result[2]
        if (result[0] == 'Topwear') and (result[1] > topwear_score):
            topwear_score = result[1]
            outputs["topwear"] = image[YMIN:YMAX, XMIN:XMAX]
        elif (result[0] == 'Bottomwear') and (result[1] > bottomwear_score):
            bottomwear_score = result[1]
            outputs["bottomwear"] = image[YMIN:YMAX, XMIN:XMAX]
        elif (result[0] == 'Footwear') and (result[1] > footwear_score):
            footwear_score = result[1]
            outputs["footwear"] = image[YMIN:YMAX, XMIN:XMAX]
    return outputs

def plot_clothes(**images):
    '''
    Plot a dictionary of Images in a row
    
    Args:
        Dict: Image Names and Images in numpy.ndarray format
    
    Returns:
        matplotlib.figure object
    '''
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.axis("off")
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
def generate_embedding(outputs, detected_objects):
    """
    Generate Embeddings for Cropped Outputs from the Object Detection Module
    
    Args:
        outputs: Dict[] objects containing Image slices for cropped detections
        detected_objects: List[] names of the detected objects like topwear, bottomwear and footwear
    
    Returns:
        outputs: Outputs with embeddings for each detected objects.
    """
    for output in detected_objects:
        image = outputs[output]
        img_path = os.path.join("temp","example.jpg")
        saved_image = Image.fromarray(image)
        saved_image.save(img_path)
        image = preprocess_image2(img_path)
        embedding = LOADED_MODELS[output](image)[0].numpy().astype(np.float32).tolist()
        outputs[output+"_embedding"] = embedding
        os.remove(img_path)
    return outputs

def __get__(csv_file, query):
    """
    Plot Top 10 similar products for each category along with the Product Link.
    """
    csv_file['distance'] = csv_file['embedding'].apply(lambda x: np.linalg.norm(np.asarray(eval(x), dtype=np.float32) - np.asarray(query, dtype=np.float32)))
    csv_file = csv_file.sort_values(by='distance').reset_index(drop=True)
    result = csv_file.iloc[:10][['product_url', 'image_url']].to_dict('records')
    for i, row in csv_file.iloc[:10].iterrows():
        plt.figure(figsize=(16,9))
        image = io.imread("..\\"+row["image_path"])
        plt.imshow(image);plt.axis("off");plt.show()
        print("Shop Now @ ", row["product_url"])
        print(254*"=")
    return result

def get_results(outputs, gender, detected_objects):
    """
    Get Similar products for a given input containing query product.
    """
    dict_results = dict()
    for output in detected_objects:
        csv_file = gender + "_" + output
        csv_file = LOADED_CSVS[csv_file].copy(deep=True)
        query = outputs[output+"_embedding"]
        dict_results[output] = __get__(csv_file, query)
    return dict_results

def final(images):
    """
    The complete pipeline for recommending similar fashion products based on a query image.
    """
    if isinstance(images, str):
        images = [images]
    if load_models() and load_CSVs():
        for img_path in images:
            inputs = preprocess_image1(img_path)
            gender_score = LOADED_MODELS["gender_classifier"](inputs)[0][0].numpy()
            gender = "mens" if  gender_score < 0.5 else "womens"
            outputs = extract_clothes(img_path)
            detected_objects = [k for k in outputs if k != "original_image"]
            plot_clothes(**outputs)
            outputs = generate_embedding(outputs, detected_objects)
            results = get_results(outputs, gender, detected_objects)

In [ ]:
img_path = "..\\google-image-search-database\\womens_casual_top_wear\\707be93f8e.jpg"
final(img_path)

In [ ]:
img_path = "..\\google-image-search-database\\mens_formal_top_wear\\3cf46ceef5.jpg"
final(img_path)

In [ ]:
img_path = "..\\google-image-search-database\\womens_casual_top_wear\\dd35ba9fdd.jpg"
final(img_path)

In [ ]:
img_path = "..\\google-image-search-database\\mens_formal_top_wear\\f6c91729cb.jpg"
final(img_path)